# Transform and Load
Brittany Lassiter

## Imports

In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [2]:
df = pd.read_csv('Data/final_tmdb_data_2010.csv.gz')
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,2004-06-25,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2 non-null      object 
 1   adult                  1 non-null      float64
 2   backdrop_path          1 non-null      object 
 3   belongs_to_collection  0 non-null      float64
 4   budget                 1 non-null      float64
 5   genres                 1 non-null      object 
 6   homepage               1 non-null      object 
 7   id                     1 non-null      float64
 8   original_language      1 non-null      object 
 9   original_title         1 non-null      object 
 10  overview               1 non-null      object 
 11  popularity             1 non-null      float64
 12  poster_path            1 non-null      object 
 13  production_companies   1 non-null      object 
 14  production_countries   1 non-null      object 
 15  release_da

In [8]:
## create a col with a list of genres
df['genres_split'] = df['genres'].str.split(',')
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genres_split
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0,"[[{'id': 10749, 'name': 'Romance'}, {'id': 1..."


In [9]:
exploded_genres = df.explode('genres_split')
exploded_genres

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genres_split
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0,[{'id': 10749
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0,'name': 'Romance'}
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0,{'id': 18
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0,'name': 'Drama'}]


In [10]:
unique_genres = sorted(exploded_genres['genres_split'].unique())

TypeError: '<' not supported between instances of 'str' and 'float'

In [11]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

NameError: name 'unique_genres' is not defined

In [12]:
## make new integer genre_id and drop string genres
df['genre_id'] = df['genres_split'].map(genre_map)
df = df.drop(columns='genres_split')

NameError: name 'genre_map' is not defined

In [13]:
## Set the dataframe index and use index=True 
df.set_index('genre_id').to_sql('genres',conn, index=True)

KeyError: "None of ['genre_id'] are in the columns"

In [14]:
## get max string length
max_str_len = df['column'].fillna('').map(len).max()

KeyError: 'column'

In [15]:
## Example
from sqlalchemy.types import *
## Calculate max string lengths for object columns
key_len = tmdb['imdb_id'].fillna('').map(len).max()
cert_len = basics['certification'].fillna('').map(len).max()
## Create a schema dictonary using Sqlalchemy datatype objects
df_schema = {
    "imdb_id": CHAR(key_len+1), 
    "budget": FLOAT(),
    "revenue": FLOAT(),
    'certification':CHAR(key_len+1)}

NameError: name 'tmdb' is not defined

In [16]:
# Save to sql with dtype and index=False
tmdb.to_sql('tmdb_data',conn, dtype=df_schema, if_exists='replace',index=False)

NameError: name 'tmdb' is not defined

In [17]:
engine.execute('ALTER TABLE tmdb_data ADD PRIMARY KEY (`imdb_id`);')

NameError: name 'engine' is not defined